In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import json

from pyspark import SparkContext
from pyspark import sql
from pyspark.sql import SQLContext

from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

In [2]:
# Create spark session
spark = SparkSession\
    .builder\
    .master("local")\
    .appName("word_count_jsonl")\
    .config("spark.some.config.option", "some-value")\
    .getOrCreate()

In [9]:
# declare data file path for read in next line
path = 'c:/users/prayt/mini_pyspark_projects/article_samples.json.gz'
# read jsonl file into spark dataframe
json_df = spark.read.json(path)
# show spark dataframe schema
json_df.printSchema()
# observe each section_texts cell is a list of text sections

root
 |-- section_texts: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- section_titles: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)



In [7]:
# collapse list of strings for each article to one long text string for each article 
samp_df =  json_df.select('section_texts').rdd.map(lambda x: ' '.join(x[0])) 
# tokenize each article



In [8]:
len(samp_rdd.take(2))

2

In [6]:
json_rdd = spark.read.json(path)

In [13]:
# Viz what records in file look like

In [14]:
import gzip

with gzip.GzipFile(path, 'r') as zipped_file:
    # read one dictionary at a time and store in data
    # declare empty counter object
    data = []
    for a_dict in zipped_file:
        data.append(json.loads(a_dict.decode('utf-8')))

In [26]:
data[575]

{'title': 'Amathus',
 'section_titles': ['Introduction',
  'History',
  'The Site and Archaeology',
  'Gallery',
  'Notes',
  'References',
  ' External links '],
 'section_texts': ["\n\n\n\n\n'''Amathus''' or '''Amathous''' () was an ancient city and one of the ancient royal cities of Cyprus until about 300 BC. Some of its impressive remains can be seen today on the southern coast in front of Agios Tychonas, about 24 miles west of Larnaca and 6 miles east of Limassol. Its ancient cult sanctuary of Aphrodite was the second most important in Cyprus, her homeland, after Paphos.\n\nArchaeological work has recently been continued at the site and many finds are exhibited in the Limassol Museum.\n",
  '===Pre-history and ancient era===\nAncient kingdoms of Cyprus\nThe pre-history of Amathus mixes myth and archaeology. Archaeology has detected human activity from the earliest Iron Age,  BC. \nThe city\'s legendary founder was Cinyras, linked with the birth of Adonis, who called the city after